<a href="https://colab.research.google.com/github/dkdranzer/Machine-Learning-projects/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df=pd.read_csv('/content/movies.csv')

In [7]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
df.shape

(4803, 24)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [10]:
df.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


In [11]:
selected_features=['genres','keywords','tagline','crew','director']

In [12]:
print(selected_features)

['genres', 'keywords', 'tagline', 'crew', 'director']


In [13]:
for feature in selected_features:
     df[feature]=df[feature].fillna('')

In [14]:
combined_features=df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['crew']+' '+df['director']

In [15]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai [{'name': 'Daniel H...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [16]:
vectorizer=TfidfVectorizer()
features=vectorizer.fit_transform(combined_features)

In [17]:
print(features)

  (0, 182731)	0.004931249916317186
  (0, 182840)	0.005659754348804982
  (0, 194170)	0.006969331154670852
  (0, 214026)	0.006498501560946915
  (0, 194546)	0.0065022923821305
  (0, 190762)	0.01342110446993744
  (0, 189515)	0.013933174153660282
  (0, 195433)	0.01033144668703155
  (0, 215713)	0.021307677017103212
  (0, 220689)	0.007873309256730639
  (0, 189862)	0.015659599295087315
  (0, 215480)	0.01342110446993744
  (0, 193614)	0.015073428143431603
  (0, 217669)	0.004464332069793697
  (0, 221608)	0.008043851514957734
  (0, 208664)	0.007788639733348239
  (0, 209314)	0.017071932283490108
  (0, 207761)	0.31356989633576055
  (0, 216242)	0.012498548014091565
  (0, 212325)	0.013661095155028807
  (0, 195914)	0.31356989633576055
  (0, 191675)	0.31971832567567743
  (0, 193113)	0.02106698823269705
  (0, 200429)	0.31356989633576055
  (0, 193114)	0.04992628926878287
  :	:
  (4802, 193938)	0.08375584049445842
  (4802, 221365)	0.14889036679091938
  (4802, 187289)	0.1462283133022879
  (4802, 187274)	0.1

In [18]:
similarity=cosine_similarity(features)
print(similarity)

[[1.         0.58384012 0.82447969 ... 0.37491278 0.1010859  0.23664754]
 [0.58384012 1.         0.59100595 ... 0.34269925 0.08891953 0.21171623]
 [0.82447969 0.59100595 1.         ... 0.39166664 0.10919079 0.24285715]
 ...
 [0.37491278 0.34269925 0.39166664 ... 1.         0.06239152 0.15472427]
 [0.1010859  0.08891953 0.10919079 ... 0.06239152 1.         0.04208946]
 [0.23664754 0.21171623 0.24285715 ... 0.15472427 0.04208946 1.        ]]


In [19]:
print(similarity.shape)

(4803, 4803)


In [20]:
# user input
movie_name=input("Enter the movie name:")

Enter the movie name:Iron man


In [21]:
list_of_titles=df['title'].tolist()
print(list_of_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [22]:
find_close_match=difflib.get_close_matches(movie_name,list_of_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [23]:
close_match=find_close_match[0]
print(close_match)

Iron Man


In [24]:
index_of_the_movie=df[df.title==close_match]['index'].values[0]
print(index_of_the_movie)

68


In [25]:
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.49044328413018523), (1, 0.42226048511165565), (2, 0.5103196806260991), (3, 0.5198471102796701), (4, 0.5018614043405836), (5, 0.48104313420296174), (6, 0.4459595650668492), (7, 0.5303022671578155), (8, 0.3816718679819639), (9, 0.5239929470340522), (10, 0.39538718543502094), (11, 0.3562792514270537), (12, 0.38446682007830185), (13, 0.41600088647943756), (14, 0.46651022580284157), (15, 0.4837114638756322), (16, 0.5240231973882624), (17, 0.4292677779348379), (18, 0.3714256464498473), (19, 0.5053808366718286), (20, 0.5162141433952535), (21, 0.337268670963234), (22, 0.5064987211800721), (23, 0.4084411218758294), (24, 0.4220167678369375), (25, 0.4631137525952464), (26, 0.535347645184884), (27, 0.5040626339604213), (28, 0.5196935566861396), (29, 0.40460281189061015), (30, 0.5298738648710651), (31, 0.5145014179303241), (32, 0.3854054124834838), (33, 0.4866312026970027), (34, 0.30323864130488204), (35, 0.47056115925329084), (36, 0.4826490610750166), (37, 0.40207704678479855), (38, 0.51958

In [26]:
len(similarity_score)

4803

In [27]:
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(68, 0.9999999999999974), (79, 0.6035068208899891), (182, 0.5451969767306482), (165, 0.5391463644301492), (654, 0.5373500252728087), (26, 0.535347645184884), (159, 0.532912554989435), (93, 0.5306909243625809), (7, 0.5303022671578155), (30, 0.5298738648710651), (112, 0.529515505104468), (303, 0.528988298012595), (412, 0.5282686834676436), (864, 0.5279688497865459), (668, 0.5278058265769628), (186, 0.5256217718717447), (1056, 0.5255928194190745), (223, 0.5255714632638531), (1430, 0.5253922811769455), (1486, 0.5247487746757293), (16, 0.5240231973882624), (9, 0.5239929470340522), (250, 0.5235232134688118), (242, 0.5224472965721262), (171, 0.5220352155242441), (212, 0.5213454909713982), (1042, 0.5212174679879434), (169, 0.5208296725034497), (275, 0.5208289520981672), (505, 0.5207821508822789), (365, 0.5207061372071442), (1054, 0.52061137115064), (828, 0.5204591918280017), (1823, 0.5198510381468067), (3, 0.5198471102796701), (338, 0.5198030149113572), (28, 0.5196935566861396), (38, 0.519589

In [28]:
print('Movies recommended for you : \n')

i=1

for movie in sorted_similar_movies:
   index=movie[0]
   title_from_index=df[df.index==index]['title'].values[0]
   if(i<21):
       print(i,'.',title_from_index)
       i+=1


Movies recommended for you : 

1 . Iron Man
2 . Iron Man 2
3 . Ant-Man
4 . Hulk
5 . Blade: Trinity
6 . Captain America: Civil War
7 . Spider-Man
8 . Terminator 3: Rise of the Machines
9 . Avengers: Age of Ultron
10 . Spider-Man 2
11 . Alexander
12 . Catwoman
13 . The Core
14 . Blade II
15 . Elektra
16 . Bad Boys II
17 . Blade
18 . The Chronicles of Riddick
19 . Cursed
20 . The Punisher
